# Use SingleCellProjection Package 

In [1]:
]activate .

  Activating project at `~/Julia`


In [2]:
using SingleCellProjections
using DataFrames
using CSV
using Statistics
using UMAP
using LinearAlgebra
using SCTransform
using PlotlyJS
using SparseArrays
using TSne
using Clustering
using JLD2

WebIO._IJuliaInit()

## Load data from A2780 Seurat Object 

In [3]:
A2780S_raw = CSV.read("/Users/jingyi/Julia/A2780S_rawdata.csv", DataFrame);
rename!(A2780S_raw, "Column1" => "Genes")

Row,Genes,A2780S_CT_S_AAACCTGCAGGGTATG,A2780S_CT_S_AAACCTGGTAGCGTCC,A2780S_CT_S_AAACCTGGTATAGGGC,A2780S_CT_S_AAACCTGGTGAAGGCT,A2780S_CT_S_AAACCTGGTGCCTTGG,A2780S_CT_S_AAACCTGTCAGTTTGG,A2780S_CT_S_AAACCTGTCATGCAAC,A2780S_CT_S_AAACGGGCAAGGTGTG,A2780S_CT_S_AAACGGGCACATCTTT,A2780S_CT_S_AAACGGGCAGGCGATA,A2780S_CT_S_AAACGGGTCCGAAGAG,A2780S_CT_S_AAACGGGTCCTTTCGG,A2780S_CT_S_AAACGGGTCTTCAACT,A2780S_CT_S_AAAGATGAGGCCCTTG,A2780S_CT_S_AAAGATGTCCGCTGTT,A2780S_CT_S_AAAGATGTCGATCCCT,A2780S_CT_S_AAAGCAACACGGTGTC,A2780S_CT_S_AAAGCAACACTTCTGC,A2780S_CT_S_AAAGCAAGTCACTTCC,A2780S_CT_S_AAAGCAAGTGCGATAG,A2780S_CT_S_AAAGCAATCCAGAAGG,A2780S_CT_S_AAAGCAATCGAGGTAG,A2780S_CT_S_AAAGTAGAGCCTTGAT,A2780S_CT_S_AAAGTAGCACATCCAA,A2780S_CT_S_AAAGTAGCAGTCAGCC,A2780S_CT_S_AAAGTAGCATCTACGA,A2780S_CT_S_AAAGTAGGTCGAACAG,A2780S_CT_S_AAAGTAGTCACGCGGT,A2780S_CT_S_AAATGCCAGTACGACG,A2780S_CT_S_AAATGCCGTTAAAGTG,A2780S_CT_S_AACACGTAGAGAACAG,A2780S_CT_S_AACACGTCAGGGTTAG,A2780S_CT_S_AACACGTCATACTCTT,A2780S_CT_S_AACACGTCATCCTAGA,A2780S_CT_S_AACACGTTCCACTGGG,A2780S_CT_S_AACCATGAGACTTGAA,A2780S_CT_S_AACCATGAGTACACCT,A2780S_CT_S_AACCATGAGTGGCACA,A2780S_CT_S_AACCATGCAGCTCCGA,A2780S_CT_S_AACCATGGTCATCCCT,A2780S_CT_S_AACCATGGTGAGGGAG,A2780S_CT_S_AACCATGGTTACGGAG,A2780S_CT_S_AACCATGTCACGAAGG,A2780S_CT_S_AACCATGTCGCATGAT,A2780S_CT_S_AACCATGTCTGCCAGG,A2780S_CT_S_AACCGCGAGATGGCGT,A2780S_CT_S_AACCGCGAGATGTTAG,A2780S_CT_S_AACCGCGAGTCAATAG,A2780S_CT_S_AACCGCGCACCGAATT,A2780S_CT_S_AACCGCGGTAAAGGAG,A2780S_CT_S_AACCGCGGTTTGGGCC,A2780S_CT_S_AACGTTGAGCTAGCCC,A2780S_CT_S_AACGTTGCACGGCCAT,A2780S_CT_S_AACGTTGCACTACAGT,A2780S_CT_S_AACGTTGCAGCTCGAC,A2780S_CT_S_AACGTTGCAGTTAACC,A2780S_CT_S_AACGTTGGTCAATACC,A2780S_CT_S_AACGTTGGTGTTAAGA,A2780S_CT_S_AACGTTGTCGCTTAGA,A2780S_CT_S_AACGTTGTCTCCCTGA,A2780S_CT_S_AACTCAGCATGTCCTC,A2780S_CT_S_AACTCAGTCAACACAC,A2780S_CT_S_AACTCAGTCCAGGGCT,A2780S_CT_S_AACTCAGTCCGTTGTC,A2780S_CT_S_AACTCCCAGCTCTCGG,A2780S_CT_S_AACTCCCAGGCAGTCA,A2780S_CT_S_AACTCCCCAGGACGTA,A2780S_CT_S_AACTCCCGTGCATCTA,A2780S_CT_S_AACTCCCTCAGCTCGG,A2780S_CT_S_AACTCTTAGAGTACCG,A2780S_CT_S_AACTCTTCAAGAAGAG,A2780S_CT_S_AACTCTTCAATGAATG,A2780S_CT_S_AACTCTTCATAGGATA,A2780S_CT_S_AACTCTTGTTCCGGCA,A2780S_CT_S_AACTCTTTCAGCATGT,A2780S_CT_S_AACTGGTAGCCAGTAG,A2780S_CT_S_AACTGGTAGTTGTCGT,A2780S_CT_S_AACTGGTCATCGGAAG,A2780S_CT_S_AACTGGTGTACCGCTG,A2780S_CT_S_AACTTTCCAGCCAATT,A2780S_CT_S_AACTTTCGTGTTAAGA,A2780S_CT_S_AACTTTCGTTAGTGGG,A2780S_CT_S_AACTTTCGTTCCCGAG,A2780S_CT_S_AACTTTCTCAGCGATT,A2780S_CT_S_AACTTTCTCAGTTGAC,A2780S_CT_S_AACTTTCTCTTGTCAT,A2780S_CT_S_AAGACCTGTACCGTAT,A2780S_CT_S_AAGACCTGTTACGACT,A2780S_CT_S_AAGACCTTCCTGCAGG,A2780S_CT_S_AAGCCGCAGGATGGAA,A2780S_CT_S_AAGCCGCCAGATGGGT,A2780S_CT_S_AAGCCGCCAGTCAGAG,A2780S_CT_S_AAGCCGCCAGTTCATG,A2780S_CT_S_AAGCCGCGTCGCATAT,A2780S_CT_S_AAGCCGCTCTAACTGG,A2780S_CT_S_AAGCCGCTCTTGTTTG,A2780S_CT_S_AAGGAGCCAATGGATA,A2780S_CT_S_AAGGAGCGTCCCTTGT,A2780S_CT_S_AAGGCAGCACCAGTTA,⋯
,String31,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,FAM138A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,OR4F5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [4]:
A2780S_meta = CSV.read("/Users/jingyi/Julia/A2780S_metadata.csv", DataFrame);
rename!(A2780S_meta, "Column1" => "Cells")

Row,Cells,orig.ident,nCount_RNA,nFeature_RNA
,String31,String15,Int64,Int64
1,A2780S_CT_S_AAACCTGCAGGGTATG,A2780S_CT_S,19540,3903
2,A2780S_CT_S_AAACCTGGTAGCGTCC,A2780S_CT_S,19408,3499
3,A2780S_CT_S_AAACCTGGTATAGGGC,A2780S_CT_S,1643,859
4,A2780S_CT_S_AAACCTGGTGAAGGCT,A2780S_CT_S,970,661
5,A2780S_CT_S_AAACCTGGTGCCTTGG,A2780S_CT_S,580,404
6,A2780S_CT_S_AAACCTGTCAGTTTGG,A2780S_CT_S,1162,680
7,A2780S_CT_S_AAACCTGTCATGCAAC,A2780S_CT_S,1408,810
8,A2780S_CT_S_AAACGGGCAAGGTGTG,A2780S_CT_S,14693,3462
9,A2780S_CT_S_AAACGGGCACATCTTT,A2780S_CT_S,6551,2232


In [5]:
matrix = Matrix(A2780S_raw[:, 2:end])

33538×3870 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0

In [6]:
matrix_sparsed = SparseArrays.sparse(matrix)

33538×3870 SparseMatrixCSC{Int64, Int64} with 6929610 stored entries:
⎡⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⎦

In [7]:
mean_val = vec(mean(matrix, dims=2))

33538-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0007751937984496124
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.020930232558139535
 ⋮
 0.0
 0.013953488372093023
 0.0
 0.0
 0.006459948320413436
 0.0
 0.00025839793281653745
 0.0
 0.0
 0.04211886304909561
 0.0
 0.0

In [8]:
var = DataFrame(Genes = copy(A2780S_raw[!,1]));
var.id = var.Genes;
var.mean = mean_val;
var

Row,Genes,id,mean
,String31,String31,Float64
1,MIR1302-2HG,MIR1302-2HG,0.0
2,FAM138A,FAM138A,0.0
3,OR4F5,OR4F5,0.0
4,AL627309.1,AL627309.1,0.000775194
5,AL627309.3,AL627309.3,0.0
6,AL627309.2,AL627309.2,0.0
7,AL627309.4,AL627309.4,0.0
8,AL732372.1,AL732372.1,0.0
9,OR4F29,OR4F29,0.0


In [9]:
obs = copy(A2780S_meta);
obs.id = obs.Cells;
obs

Row,Cells,orig.ident,nCount_RNA,nFeature_RNA,id
,String31,String15,Int64,Int64,String31
1,A2780S_CT_S_AAACCTGCAGGGTATG,A2780S_CT_S,19540,3903,A2780S_CT_S_AAACCTGCAGGGTATG
2,A2780S_CT_S_AAACCTGGTAGCGTCC,A2780S_CT_S,19408,3499,A2780S_CT_S_AAACCTGGTAGCGTCC
3,A2780S_CT_S_AAACCTGGTATAGGGC,A2780S_CT_S,1643,859,A2780S_CT_S_AAACCTGGTATAGGGC
4,A2780S_CT_S_AAACCTGGTGAAGGCT,A2780S_CT_S,970,661,A2780S_CT_S_AAACCTGGTGAAGGCT
5,A2780S_CT_S_AAACCTGGTGCCTTGG,A2780S_CT_S,580,404,A2780S_CT_S_AAACCTGGTGCCTTGG
6,A2780S_CT_S_AAACCTGTCAGTTTGG,A2780S_CT_S,1162,680,A2780S_CT_S_AAACCTGTCAGTTTGG
7,A2780S_CT_S_AAACCTGTCATGCAAC,A2780S_CT_S,1408,810,A2780S_CT_S_AAACCTGTCATGCAAC
8,A2780S_CT_S_AAACGGGCAAGGTGTG,A2780S_CT_S,14693,3462,A2780S_CT_S_AAACGGGCAAGGTGTG
9,A2780S_CT_S_AAACGGGCACATCTTT,A2780S_CT_S,6551,2232,A2780S_CT_S_AAACGGGCACATCTTT


In [10]:
A2780S = DataMatrix(matrix_sparsed, var, obs; var_id_cols = nothing, obs_id_cols = nothing)

DataMatrix (33538 variables and 3870 observations)
  SparseMatrixCSC{Int64, Int64}
  Variables: id, Genes, mean
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA

## QC

In [11]:
var_counts_fraction!(A2780S, "Genes"=>startswith(r"MT-"), Returns(true), "fraction_mt")

DataMatrix (33538 variables and 3870 observations)
  SparseMatrixCSC{Int64, Int64}
  Variables: id, Genes, mean
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: VarCountsFractionModel(subset_size=13, total_size=33538, col="fraction_mt")

## Filter

In [12]:
describe(A2780S.obs[!,:nFeature_RNA])

Summary Stats:
Length:         3870
Missing Count:  0
Mean:           1790.596899
Minimum:        123.000000
1st Quartile:   575.000000
Median:         1095.000000
3rd Quartile:   3235.750000
Maximum:        9602.000000
Type:           Int64


In [13]:
@show mean_feature = mean(A2780S.obs[!,:nFeature_RNA]);
@show sd_feature = std(A2780S.obs[!,:nFeature_RNA]);
@show nFeature_Lower = 200;
@show nFeature_Upper = mean_feature + 2 * sd_feature;

mean_feature = mean(A2780S.obs[!, :nFeature_RNA]) = 1790.5968992248063
sd_feature = std(A2780S.obs[!, :nFeature_RNA]) = 1459.8241111706327
nFeature_Lower = 200 = 200
nFeature_Upper = mean_feature + 2sd_feature = 4710.245121566071


In [14]:
describe(A2780S.obs[!,:fraction_mt])

Summary Stats:
Length:         3870
Missing Count:  0
Mean:           0.131966
Minimum:        0.000000
1st Quartile:   0.033962
Median:         0.067723
3rd Quartile:   0.098820
Maximum:        0.929275
Type:           Float64


In [15]:
@show mean_fractionMT = mean(A2780S.obs[!,:fraction_mt]);
@show sd_fractionMT = std(A2780S.obs[!,:fraction_mt]);
@show fractionMito_Upper = mean_fractionMT + 2 * sd_fractionMT;
@show fractionMito_Low = mean_fractionMT - 2 * sd_fractionMT;

mean_fractionMT = mean(A2780S.obs[!, :fraction_mt]) = 0.13196564783999243
sd_fractionMT = std(A2780S.obs[!, :fraction_mt]) = 0.18480269138043545
fractionMito_Upper = mean_fractionMT + 2sd_fractionMT = 0.5015710306008634
fractionMito_Low = mean_fractionMT - 2sd_fractionMT = -0.23763973492087848


In [16]:
A2780S_filtered_obs = filter_obs(row -> 
                         row[:nFeature_RNA] > nFeature_Lower &&
                         row[:nFeature_RNA] < nFeature_Upper &&
                         row[:fraction_mt] < fractionMito_Upper, A2780S);
A2780S_filtered = filter_var(row -> 0.0125 <= row.mean <= 3.0 , A2780S_filtered_obs)

DataMatrix (10475 variables and 3449 observations)
  SparseMatrixCSC{Int64, Int64}
  Variables: id, Genes, mean
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: FilterModel([13, 16, 18, 20, 21, 22, 23, 27, 28, 30  …  33494, 33496, 33501, 33505, 33508, 33512, 33524, 33526, 33528, 33536], :), Filter, VarCountsFraction

## Transformed

In [17]:
A2780S_filtered_transformed = sctransform(A2780S_filtered)

[ Info: SCTransform parameters loaded from cache.
[ Info: - Removed 2 variables that were not found in Model


DataMatrix (10473 variables and 3449 observations)
  A+B₁B₂B₃
  Variables: id, Genes, mean, logGeneMean, outlier, beta0, beta1, theta
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: SCTransformModel(nvar=10473, clip=10.72), Filter, Filter, VarCountsFraction

## Normalization

In [18]:
A2780S_filtered_transformed_normalized = normalize_matrix(A2780S_filtered_transformed,"fraction_mt","nCount_RNA")

DataMatrix (10473 variables and 3449 observations)
  A+B₁B₂B₃+(-β)X'
  Variables: id, Genes, mean, logGeneMean, outlier, beta0, beta1, theta
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: NormalizationModel(rank=3, ~1+num(fraction_mt)+num(nCount_RNA)), SCTransform, Filter, Filter, VarCountsFraction

## PCA

In [19]:
A2780S_reduced = svd(A2780S_filtered_transformed_normalized; nsv=200)

DataMatrix (10473 variables and 3449 observations)
  SVD (200 dimensions)
  Variables: id, Genes, mean, logGeneMean, outlier, beta0, beta1, theta
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: SVDModel(nsv=200), Normalization, SCTransform, Filter, Filter, ...

## Plot function

In [20]:
function plot_categorical_3d(data, annotation; marker_size=7)
    points = obs_coordinates(data)
    traces = GenericTrace[]
    for sub in groupby(data.obs, annotation; sort=true)
        value = sub[1,annotation]
        ind = parentindices(sub)[1]
        push!(traces, scatter3d(;x=points[1,ind], y=points[2,ind], z=points[3,ind], mode="markers", marker_size, name=value))
    end
    plot(traces, Layout(;legend=attr(itemsizing="constant")))
end

plot_categorical_3d (generic function with 1 method)

In [21]:
pca = plot_categorical_3d(A2780S_reduced, "orig.ident")
savefig(pca, "SingleCellProjection/pca.html")

"SingleCellProjection/pca.html"

## UMAP

In [23]:
A2780S_umapped = umap(A2780S_reduced, 50 )

DataMatrix (50 variables and 3449 observations)
  Matrix{Float64}
  Variables: id
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: UMAPModel(n_components=50), SVD, Normalization, SCTransform, Filter, ...

In [24]:
mapped = plot_categorical_3d(A2780S_umapped, "orig.ident")
savefig(mapped, "SingleCellProjection/umapped.html")

"SingleCellProjection/umapped.html"

## T-SNE

In [25]:
T = tsne(A2780S_reduced,50)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/dMfiC/src/ProgressMeter.jl:594
Computing t-SNE 100%|████████████████████████████████████| Time: 0:00:32
  KL_divergence:  2.6595


DataMatrix (50 variables and 3449 observations)
  Matrix{Float64}
  Variables: id
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: NearestNeighborModel(base="tsne", k=10), SVD, Normalization, SCTransform, Filter, ...

In [26]:
tsne_plot = plot_categorical_3d(A2780S_umapped, "orig.ident")
savefig(tsne_plot, "SingleCellProjection/tsne.html")

"SingleCellProjection/tsne.html"

## force_layout graph

In [27]:
A2780S_fl = force_layout(A2780S_reduced; ndim = 10, k = 100)

DataMatrix (10 variables and 3449 observations)
  Matrix{Float64}
  Variables: id
  Observations: id, Cells, orig.ident, nCount_RNA, nFeature_RNA, fraction_mt
  Models: NearestNeighborModel(base="force_layout", k=10), SVD, Normalization, SCTransform, Filter, ...

In [28]:
fl_plot = plot_categorical_3d(A2780S_fl, "orig.ident")
savefig(fl_plot, "SingleCellProjection/force_layout.html")

"SingleCellProjection/force_layout.html"